In [1]:
require 'nn'
require 'hdf5'

local Squeeze, parent = torch.class('nn.Squeeze', 'nn.Module')

function Squeeze:updateOutput(input)
    self.size = input:size()
    self.output = input:squeeze()
    return self.output
end

function Squeeze:updateGradInput(input, gradOutput)
  self.gradInput = gradOutput:view(self.size)
  return self.gradInput  
end

In [17]:
myFile = hdf5.open('2-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [18]:
data

{
  valid_output : LongTensor - size: 3370x50
  train : LongTensor - size: 259471x3
  nwords : LongTensor - size: 1
  valid : LongTensor - size: 3370x51
  test : LongTensor - size: 3761x51
}


In [19]:
train = data['train']:narrow(2,1,5)

In [20]:
train[1]

   14
    5
 7475
[torch.LongTensor of size 3]



In [14]:
train_input = train:narrow(2,1,4)
train_output = train:narrow(2,5,1)

In [15]:
LT = nn.LookupTable(data['nwords'][1],50)

In [16]:
LT:forward(train_input[1])

Columns 1 to 10
-0.5626 -0.7167 -0.8011  0.1635  0.4544 -2.0875 -0.4934  0.4453  0.4442 -0.8109
-0.5626 -0.7167 -0.8011  0.1635  0.4544 -2.0875 -0.4934  0.4453  0.4442 -0.8109
-0.5626 -0.7167 -0.8011  0.1635  0.4544 -2.0875 -0.4934  0.4453  0.4442 -0.8109
-0.5626 -0.7167 -0.8011  0.1635  0.4544 -2.0875 -0.4934  0.4453  0.4442 -0.8109

Columns 11 to 20
-0.5965  0.5523  0.7668 -0.8587 -1.5801 -0.4235 -0.4902 -0.7762 -0.1874  0.0707
-0.5965  0.5523  0.7668 -0.8587 -1.5801 -0.4235 -0.4902 -0.7762 -0.1874  0.0707
-0.5965  0.5523  0.7668 -0.8587 -1.5801 -0.4235 -0.4902 -0.7762 -0.1874  0.0707
-0.5965  0.5523  0.7668 -0.8587 -1.5801 -0.4235 -0.4902 -0.7762 -0.1874  0.0707

Columns 21 to 30
-0.9597 -0.9056 -1.4552 -0.2263 -0.1127 -0.2337  0.9416  1.6251 -1.7943 -0.8021
-0.9597 -0.9056 -1.4552 -0.2263 -0.1127 -0.2337  0.9416  1.6251 -1.7943 -0.8021
-0.9597 -0.9056 -1.4552 -0.2263 -0.1127 -0.2337  0.9416  1.6251 -1.7943 -0.8021
-0.9597 -0.9056 -1.4552 -0.2263 -0.1127 -0.2337  0.9416  1.6251 -1.7

In [8]:
nnlm = nn.Sequential()
nnlm:add(nn.LookupTable(data['nwords'][1],50))
neuralnet:add(nn.View(1,-1,5*50))
neuralnet:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet:add(nn.Linear(5*dim_hidden,dim_hidden2))
neuralnet:add(nn.HardTanh())
neuralnet:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet:add(nn.LogSoftMax())

[string "nnlm = nn.Sequential()..."]:3: attempt to index global 'neuralnet' (a nil value)
stack traceback:
	[string "nnlm = nn.Sequential()..."]:3: in main chunk
	[C]: in function 'xpcall'
	.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:179: in function <.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:143>
	.../virgileaudi/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x01090fbb50: 

In [ ]:
criterion = nn.ClassNLLCriterion()

In [ ]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [ ]:
trainer = nn.StochasticGradient(neuralnet, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 10
trainer:train(dataset)